
# Intro to LangChain

LangChain is an open-source framework that gives developers the tools they need to create applications using large language models (LLMs). In its essence, LangChain is a prompt orchestration tool that makes it easier for teams to connect various prompts interactively.

In [1]:
#!pip install -qU openai
#!pip install -qU langchain
#!pip install -qU langchain-openai
#!pip install -qU tiktoken

!pip install pydantic=="1.10.12"

# Accessing OpenAI Directly

In [2]:
import os
import openai

#os.environ["OPENAI_API_KEY"] = "<the key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

client = openai.OpenAI()
model="gpt-3.5-turbo"
#model="gpt-4"

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},
        ]
    response = client.chat.completions.create(model=model,
                                              messages=messages,
                                              temperature=2
                                              )
    return response.choices[0].message.content

## Simple Query

In [3]:
response = get_completion("Can you tell me how much is 1+1?")
print(response)

Certainly! 1+1 is equal to 2. 


## Queries with custom prompts using formatted strings

In [4]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [5]:
response = get_completion(prompt)
print(response)

I am upset that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add to that, the warranty does not include the cost of cleaning up my kitchen. I need your help, please!



## Why Should we use a framework to wrap the direct calls to OpenAI?    

The LangChain pipelines consist of the following modules:

+ **Models**: Models mostly cover Large Language models. A large language model of considerable size is a model that comprises a neural network with numerous parameters and is trained on vast quantities of unlabeled text  
+ **Prompts**: prompt is the input that we give to any system to refine our answers to make them more accurate or more specific according to our use case. Many times you may want to get more structured information than just text back. We can use Prompts in conjunction with Parsers
+ **Parsers**: Output parsers are responsible for taking the output of an LLM and transforming it to a more suitable format. This is very useful when you are using LLMs to generate any form of structured data.  
+ **Memory**: Chains and Agents in LangChain operate in a stateless mode by default, meaning that they handle each incoming query independently. However, there are certain applications, like chatbots, where it is of great importance to retain previous interactions, both over the short and long term. This is where the concept of “Memory” comes into play.  
+ **Chains**: Chains provide a means to merge various components into a unified application. A chain can be created, for instance, that receives input from a user, formats it using a PromptTemplate, and subsequently transmits the formatted reply to an LLM. More intricate chains can be generated by integrating multiple chains with other components.  
+ **Agents**: Certain applications may necessitate not only a pre-determined sequence of LLM/other tool calls but also an uncertain sequence that is dependent on the user’s input. These kinds of sequences include an “agent” that has access to a range of tools. Based on the user input, the agent may determine which of these tools, if any, should be called.  
+ **Callbacks**: Allow you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks.  
+ **Indexes**: Indexes are information stored in local databases that can be used for Augment the capabilities of the models being used. We will see them in action with RAG pipelines  

Let's wrap the pipeline:

![](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*05zEoeNU7DVYOFzjugiF_w.jpeg)


# I - Simple Pipelines with Langchain  

In [6]:
#os.environ["OPENAI_API_KEY"] = "<the key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

llm="gpt-3.5-turbo"

from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=llm)

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder

from langchain_core.messages import SystemMessage
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser

from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationTokenBufferMemory
from langchain.memory import ConversationSummaryBufferMemory

from langchain.chains import ConversationChain

Once you've installed and initialized the LLM of your choice, we could use it!  Let's ask it some question:

In [8]:
model.invoke("Why is the sky blue?")

AIMessage(content="The sky appears blue because of the way Earth's atmosphere scatters sunlight. Sunlight is made up of all the colors of the rainbow, but blue light is scattered more than other colors because it travels as shorter, smaller waves. When sunlight reaches Earth's atmosphere, the blue light is scattered in all directions by the gases and particles in the air. This is why we see the sky as blue during the day.", response_metadata={'token_usage': {'completion_tokens': 83, 'prompt_tokens': 13, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8b139933-09ef-4291-b495-e447ee2d0d85-0')

Prompt templates convert raw user input to better input to the LLM.

In [9]:
prompt = ChatPromptTemplate.from_messages([("system", "You are a first grade teacher."),
                                           ("user", "{input}")
                                         ])

chain = prompt | model
chain.invoke({"input": "Why is the sky blue?"})

AIMessage(content="That's a great question! The sky appears blue because of the way our Earth's atmosphere scatters sunlight. When sunlight reaches the Earth's atmosphere, it is made up of different colors, with blue light being scattered more than the other colors because it travels in shorter, smaller waves. This scattered blue light is what we see when we look up at the sky during the day.", response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 24, 'total_tokens': 100}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b128213c-e09a-4849-bd0e-cedddd714bec-0')

The output of a ChatModel (and therefore, of this chain) is a message object.  
However, it's often much more convenient to work with strings.  
Let's add a simple output parser to convert the chat message to a string.

In [10]:
output_parser = StrOutputParser()

In [11]:
chain = prompt | model | output_parser
chain.invoke({"input": "Why is the sky blue?"})

"That's a great question! The sky appears blue because of the way that sunlight interacts with the Earth's atmosphere. When sunlight reaches the Earth, it is made up of different colors, including red, orange, yellow, green, blue, indigo, and violet. These colors are all part of the visible light spectrum.\n\nThe Earth's atmosphere is made up of gases and particles. When sunlight enters the atmosphere, the shorter blue wavelengths of light are scattered in all directions by the gases and particles in the air. This is called Rayleigh scattering.\n\nSince blue light is scattered more than the other colors, it is scattered in all directions and makes the sky look blue to our eyes. This is why we see the sky as blue during the day."

# II - Exploring Chain Elements - Langchain Expression Language

Notice this line of the code, where we piece together these different components into a single chain using LCEL:  

**chain = prompt | model | output_parser**

## 1. [Prompts](https://python.langchain.com/docs/modules/model_io/prompts/)    
prompt is a BasePromptTemplate, which means it takes in a dictionary of template variables and produces a PromptValue.  
+ A PromptValue is a wrapper around a completed prompt that can be passed to either a LLM (which takes a string as input) or ChatModel (which takes a sequence of messages as input).  
+ It can work with either language model type because it defines logic both for producing BaseMessages and for producing a string.

In [12]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
chain = prompt | model | output_parser
chain.invoke({"topic": "chicken"})

'Why did the chicken join a band? Because it had the drumsticks!'

In [13]:
prompt_value = prompt.invoke({"topic": "ice cream"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [14]:
prompt_value.to_messages()

[HumanMessage(content='tell me a short joke about ice cream')]

In [15]:
prompt_value.to_string()

'Human: tell me a short joke about ice cream'

In [16]:
prompt = ChatPromptTemplate.from_messages([("system", "You are a first grade teacher."),
                                           ("user", "{input}")
                                         ])
chain = prompt | model | output_parser
chain.invoke({"input": "Why is the sky blue?"})

"That's a great question! The sky appears blue because of the way that sunlight interacts with the Earth's atmosphere. When sunlight reaches the Earth's atmosphere, it is scattered in all directions by the gases and particles in the air. Blue light is scattered more than other colors because it travels as shorter, smaller waves. This is why we see the sky as blue during the day!"

## 2. [Models](https://python.langchain.com/docs/modules/model_io/chat/quick_start/)    
The PromptValue is then passed to model.  
In this case our model is a ChatModel, meaning it will output a BaseMessage.

In [17]:
message = model.invoke(prompt_value)
message

AIMessage(content='Why did the ice cream truck break down? Because it had too many sundaes!', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 15, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-97fc72c3-ecf6-4507-a45b-742b6df921f3-0')

If our model was an pure LLM, it would output a string.

In [18]:
llm2 = OpenAI(model="gpt-3.5-turbo-instruct")
llm2.invoke(prompt_value)

'\n\nRobot: Why did the ice cream truck break down? Because it had a fudge-ical!'

## 3. [Output parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)    
And lastly we pass our model output to the output_parser, which is a BaseOutputParser meaning it takes either a string or a BaseMessage as input. The specific StrOutputParser simply converts any input into a string.

In [19]:
output_parser.invoke(message)

'Why did the ice cream truck break down? Because it had too many sundaes!'

## 4. [Chains](https://python.langchain.com/v0.1/docs/modules/chains/)  

Chains are combination of steps. We have followed the steps along:

+ We passed the user input on the desired topic as {"topic": "ice cream"}
+ The prompt component takes the user input, which is then used to construct a PromptValue after using the topic to construct the prompt.
+ The model component takes the generated prompt, and passes into the OpenAI LLM model for evaluation.
+ The generated output from the model is a ChatMessage object.
+ Finally, the output_parser component takes in a ChatMessage, and transforms this into a Python string, which is returned from the invoke method.

Langchain is currently revamping all the chain methods to use the [LCEL](https://python.langchain.com/v0.1/docs/expression_language/). That is why we are not exploring all types of chains in this notebook.  , 

# 5. [Types of Prompts](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/)  
Let's explore some other types of Prompts  

## 5.1 PromptTemplate

Use PromptTemplate to create a template for a string prompt.
By default, PromptTemplate uses Python’s str.format syntax for templating.

In [20]:
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [21]:
prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

## 5.2 ChatPromptTemplate

The prompt to chat models/ is a list of chat messages.  
Each chat message is associated with content, and an additional parameter called role. For example, in the OpenAI Chat Completions API, a chat message can be associated with an AI assistant, a human or a system role.

In [22]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.'),
 HumanMessage(content='Hello, how are you doing?'),
 AIMessage(content="I'm doing well, thanks!"),
 HumanMessage(content='What is your name?')]

This is equivalent as doing the following, directly with OpenAI

In [23]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful AI bot. Your name is Bob."},
        {"role": "user", "content": "Hello, how are you doing?"},
        {"role": "assistant", "content": "I'm doing well, thanks!"},
        {"role": "user", "content": "What is your name?"},
    ],
)

The ChatPromptTemplate.from_messages static method accepts a variety of message representations and is a convenient way to format input to chat models with exactly the messages you want.  

For example, in addition to using the 2-tuple representation of (type, content) used above, you could pass in an instance of MessagePromptTemplate or BaseMessage.

In [24]:
chat_template = ChatPromptTemplate.from_messages(
    [SystemMessage(content=("You are a helpful assistant that re-writes the user's text to "
                            "sound more upbeat.")),
     HumanMessagePromptTemplate.from_template("{text}"),
    ])

messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


## 5.3 Message Prompts  
LangChain provides different types of MessagePromptTemplate. The most commonly used are
+ AIMessagePromptTemplate  
+ SystemMessagePromptTemplate  
+ HumanMessagePromptTemplate  

Which create an AI message, system message and human message respectively.  
In cases where the chat model supports taking chat message with arbitrary role, you can use ChatMessagePromptTemplate, which allows user to specify the role name.  
https://python.langchain.com/docs/modules/model_io/chat/message_types/  

In [25]:
prompt = "May the {subject} be with you"
chat_message_prompt = ChatMessagePromptTemplate.from_template(role="Jedi", template=prompt)
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', role='Jedi')

# 5.4 MessagesPlaceholder  
LangChain also provides MessagesPlaceholder, which gives you full control of what messages to be rendered during formatting. This can be useful when you are uncertain of what role you should be using for your message prompt templates or when you wish to insert a list of messages during formatting.

In [26]:
human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)
chat_prompt = ChatPromptTemplate.from_messages([MessagesPlaceholder(variable_name="conversation"),
                                                human_message_template])
human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.
2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.
3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count="10").to_messages()

[HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience'),
 HumanMessage(content='Summarize our conversation so far in 10 words.')]

# 6 - [Types of Output Parsers](https://python.langchain.com/docs/modules/model_io/output_parsers/)  

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in. Output parsers are classes that help structure language model responses.  
There are two main methods an output parser must implement:  
+ “Get format instructions”: A method which returns a string containing instructions for how the output of a language model should be formatted.
+ “Parse”: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

And then one optional one:

+ “Parse with prompt”: A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to be the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.

Let's explore the **CSV Parser**

In [27]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(template="List five {subject}.\n{format_instructions}",
                        input_variables=["subject"],
                        partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

chain = prompt | model | output_parser
a = chain.invoke({"subject": "ice cream flavors"})

In [28]:
for s in chain.stream({"subject": "ice cream flavors"}):
    print(s)

['Vanilla']
['Chocolate']
['Strawberry']
['Mint Chocolate Chip']
['Cookies and Cream']


Let's take a look at the **Pandas Dataframe Parser**  

In [29]:
import pprint
from typing import Any, Dict
import pandas as pd

In [30]:
# Solely for documentation purposes.
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    for key in parser_output.keys():
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output)

Define your desired Pandas DataFrame and set up a parser + inject instructions into the prompt template.

In [31]:

df = pd.DataFrame(
    {
        "num_legs": [2, 4, 8, 0],
        "num_wings": [2, 0, 0, 0],
        "num_specimen_seen": [10, 2, 1, 8],
    }
)

parser = PandasDataFrameOutputParser(dataframe=df)

Here's an example of a column operation being performed.

In [32]:
df_query = "Retrieve the num_wings column."
prompt = PromptTemplate(template="Answer the user query.\n{format_instructions}\n{query}\n",
                        input_variables=["query"],
                        partial_variables={"format_instructions": parser.get_format_instructions()},
                        )

chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})
format_parser_output(parser_output)

{'num_wings': {0: 2,
               1: 0,
               2: 0,
               3: 0}}


Here's an example of a row operation being performed.

In [33]:
df_query = "Retrieve the first row."
prompt = PromptTemplate(template="Answer the user query.\n{format_instructions}\n{query}\n",
                        input_variables=["query"],
                        partial_variables={"format_instructions": parser.get_format_instructions()},
                        )
chain = prompt | model | parser
parser_output = chain.invoke({"query": df_query})
format_parser_output(parser_output)

{'0': {'num_legs': 2,
       'num_specimen_seen': 10,
       'num_wings': 2}}


Below we go over a more powerful output parser, the **PydanticOutputParser**.

a) Define your desired data structure.

In [34]:

class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field

b) Set up a parser + inject instructions into the prompt template.

In [35]:
parser = PydanticOutputParser(pydantic_object=Joke)
prompt = PromptTemplate(template="Answer the user query.\n{format_instructions}\n{query}\n",
                        input_variables=["query"],
                        partial_variables={"format_instructions": parser.get_format_instructions()},
                        )

c) And a query intended to prompt a language model to populate the data structure.

In [36]:
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

Joke(setup="Why couldn't the bicycle stand up by itself?", punchline='Because it was two tired!')

In [37]:
json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

# 7 - [Memory](https://python.langchain.com/v0.1/docs/modules/memory/)  

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

### Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQcYZAK7bD8PPvjbAIe5tLk19SX9zKaSGHlVrE_vKrDk09tCgj7ujp_re7SpYHI8I7yUA&usqp=CAU) 

## 7.1 - ConversationBufferMemory  

We are going to use the simplest memory type.  
Let's check what is happening behind the scenes using the ```verbose``` mode of the LLM:  

In [38]:
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=model, memory=memory, verbose=True)
conversation.predict(input="Hi, my name is Renato")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Renato
AI:

> Finished chain.


"Hello Renato! It's nice to meet you. How can I assist you today?"

In [39]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Renato
AI: Hello Renato! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1 + 1 equals 2. Is there anything else you would like to know?'

In [40]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Renato
AI: Hello Renato! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Renato.'

##### As we are using a memory module, we can check what is stored in there:  

In [41]:
print(memory.buffer)

Human: Hi, my name is Renato
AI: Hello Renato! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Renato.


In [42]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Renato\nAI: Hello Renato! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1 + 1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Renato."}

##### Now let's erase the memory buffer and insert something else in it:  

In [43]:
memory = ConversationBufferMemory()

memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

print(memory.buffer)

Human: Hi
AI: What's up


In [44]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

##### Let's add even more context:  

In [45]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

##### Now, let's check if the model still know the information from a previous interaction:  

In [46]:
conversation = ConversationChain(llm=model,
                                 memory=memory,
                                 verbose=True
                                 )
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What is my name?
AI:

> Finished chain.


"I'm sorry, I do not have access to your personal information, so I do not know your name."

## ConversationBufferWindowMemory  

We can define a buffer size to determine how much context will be kept:  

In [47]:
memory = ConversationBufferWindowMemory(k=1)

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

##### We see that only one interaction was saved. Now let's see how it would be with the LLM:  

In [48]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(llm=model, memory=memory, verbose=False)
conversation.predict(input="Hi, my name is Renato")

"Hello Renato! It's nice to meet you. How can I assist you today?"

In [49]:
conversation.predict(input="What is 1+1?")

'1+1 equals 2. Is there anything else you would like to know?'

In [50]:
conversation.predict(input="What is my name?")

"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?"

## ConversationTokenBufferMemory

Similarly, we can define a buffer size in the quantity of tokens:  

In [51]:
memory = ConversationTokenBufferMemory(llm=model, max_token_limit=30)

memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})

memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Something Beautiful!"})

memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

memory.load_memory_variables({})

{'history': 'AI: Something Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory  

This last type of memory creates an automatic summary of the previous interactions to store:  

In [52]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=model, max_token_limit=100)

memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to show the latest LLM demo during the lunch meeting.'}

In [53]:
conversation = ConversationChain(llm=model, memory=memory, verbose=True)
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to show the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


"For the LLM demo, it would be great to showcase the latest language model capabilities, such as text generation, language translation, and sentiment analysis. You could also highlight the model's accuracy and efficiency compared to other existing models in the market. Additionally, demonstrating how the model can be easily integrated into different applications or platforms would be impressive to the customer."

In [54]:
memory.load_memory_variables({})

{'history': "System: The human and AI exchange greetings and discuss the schedule for the day. The AI informs the human of a morning meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human is advised to bring their laptop to show the latest LLM demo during the lunch meeting.\nHuman: What would be a good demo to show?\nAI: For the LLM demo, it would be great to showcase the latest language model capabilities, such as text generation, language translation, and sentiment analysis. You could also highlight the model's accuracy and efficiency compared to other existing models in the market. Additionally, demonstrating how the model can be easily integrated into different applications or platforms would be impressive to the customer."}